# The Currencies data-set

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/Currencies.ipynb)

In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(RPostgres)
library(plutoR)
options("scipen"=999)
source("config.R")

#initialize
currencies <- Currencies()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### get traded futures pairs

In [2]:
currencies$NseFuturesTimeSeries() %>%
  group_by(SYMBOL) %>%
  summarize(START_DT = min(TIME_STAMP), END_DT = max(TIME_STAMP)) %>%
  arrange(START_DT) %>%
  print()

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:     lazy query [?? x 3]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Ordered by: START_DT
  SYMBOL START_DT   END_DT    
  <chr>  <date>     <date>    
1 GBPINR 2010-10-29 2019-07-22
2 USDINR 2010-10-29 2019-07-22
3 JPYINR 2010-10-29 2019-07-22
4 EURINR 2010-10-29 2019-07-22
5 USDJPY 2018-02-27 2019-07-22
6 GBPUSD 2018-02-27 2019-07-22
7 EURUSD 2018-02-27 2019-07-22


### get the latest USDINR option chain for the nearest expiry

In [3]:
maxDt <- (currencies$NseOptionsTimeSeries() %>%
            summarize(MAX_DT = max(TIME_STAMP)) %>%
            collect())$MAX_DT[[1]]

expiryDt <- (currencies$NseOptionsTimeSeries() %>%
            filter(TIME_STAMP == maxDt & SYMBOL == 'USDINR') %>%
            summarize(MAX_DT = max(EXPIRY)) %>%
            collect())$MAX_DT[[1]]

currencies$NseOptionsTimeSeries() %>%
  filter(TIME_STAMP == maxDt & SYMBOL == 'USDINR' & EXPIRY == expiryDt) %>%
  arrange(STRIKE, OTYPE) %>%
  print()

# Source:     lazy query [?? x 13]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Ordered by: STRIKE, OTYPE
   TIME_STAMP SYMBOL EXPIRY     STRIKE OTYPE  HIGH   LOW  OPEN CLOSE SETTLE
   <date>     <chr>  <date>      <dbl> <chr> <dbl> <dbl> <dbl> <dbl>  <dbl>
 1 2019-07-22 USDINR 2020-06-26   65.5 CE        0     0     0    NA 6.20  
 2 2019-07-22 USDINR 2020-06-26   65.5 PE        0     0     0    NA 0.0325
 3 2019-07-22 USDINR 2020-06-26   65.8 CE        0     0     0    NA 5.97  
 4 2019-07-22 USDINR 2020-06-26   65.8 PE        0     0     0    NA 0.04  
 5 2019-07-22 USDINR 2020-06-26   66   CE        0     0     0    NA 5.74  
 6 2019-07-22 USDINR 2020-06-26   66   PE        0     0     0    NA 0.0475
 7 2019-07-22 USDINR 2020-06-26   66.2 CE        0     0     0    NA 5.51  
 8 2019-07-22 USDINR 2020-06-26   66.2 PE        0     0     0    NA 0.0575
 9 2019-07-22 USDINR 2020-06-26   66.5 CE        0     0     0    NA 5.28  
10 2019-07-22 USDINR 2020-06-26   

### get the currencies tracked by AlphaVantage end-of-day

In [4]:
currencies$AvEodTimeSeries() %>%
  group_by(SYMBOL) %>%
  summarize(START_DT = min(TIME_STAMP), END_DT = max(TIME_STAMP)) %>%
  arrange(START_DT) %>%
  print(n=Inf)

# Source:     lazy query [?? x 3]
# Database:   postgres [ro1@WINDOWS:5432/StockVizBeka]
# Ordered by: START_DT
    SYMBOL START_DT   END_DT    
    <chr>  <date>     <date>    
  1 MYR    1998-01-23 2019-07-23
  2 TWD    2000-04-24 2019-07-23
  3 JPY    2000-04-27 2019-07-23
  4 CHF    2000-07-19 2019-07-23
  5 CAD    2000-07-27 2019-07-23
  6 HKD    2000-08-04 2019-07-23
  7 SGD    2000-08-10 2019-07-23
  8 CNY    2005-11-09 2019-07-23
  9 MOP    2011-04-07 2019-07-23
 10 BHD    2014-11-07 2019-07-23
 11 BIF    2014-11-07 2019-07-23
 12 BND    2014-11-07 2019-07-23
 13 BOB    2014-11-07 2019-07-23
 14 BRL    2014-11-07 2019-07-23
 15 BSD    2014-11-07 2019-07-23
 16 BWP    2014-11-07 2019-07-22
 17 BZD    2014-11-07 2019-07-23
 18 CDF    2014-11-07 2019-07-23
 19 CLF    2014-11-07 2019-07-22
 20 CLP    2014-11-07 2019-07-23
 21 CNH    2014-11-07 2019-07-23
 22 ZWL    2014-11-07 2015-10-15
 23 AFN    2014-11-07 2019-07-23
 24 ALL    2014-11-07 2019-07-22
 25 ANG    2014-11-07 2019-07-

### get the currencies tracked by AlphaVantage 30-min bars 

In [5]:
currencies$Av30minTimeSeries() %>%
  group_by(SYMBOL) %>%
  summarize(START_DT = min(TIME_STAMP), END_DT = max(TIME_STAMP), COUNT = n()) %>%
  arrange(START_DT) %>%
  print(n=Inf)

# Source:     lazy query [?? x 4]
# Database:   postgres [ro1@WINDOWS:5432/StockVizBeka]
# Ordered by: START_DT
    SYMBOL START_DT            END_DT              COUNT     
    <chr>  <dttm>              <dttm>              <integr64>
  1 CNY    2018-08-03 02:00:00 2019-07-23 11:00:00  5675     
  2 AFN    2018-08-21 16:00:00 2019-07-23 04:00:00 14354     
  3 AED    2018-08-21 16:00:00 2019-07-23 06:30:00 14362     
  4 DZD    2018-08-21 16:00:00 2019-07-23 07:00:00 13091     
  5 EGP    2018-08-21 16:00:00 2019-07-23 04:00:00 13110     
  6 ERN    2018-08-21 16:00:00 2019-07-23 04:00:00 13097     
  7 GHS    2018-08-21 16:00:00 2019-07-23 04:00:00 13082     
  8 GMD    2018-08-21 16:00:00 2019-07-22 13:30:00 10586     
  9 GNF    2018-08-21 16:00:00 2019-07-23 04:00:00 13096     
 10 GTQ    2018-08-21 16:00:00 2019-07-23 07:30:00 13103     
 11 GYD    2018-08-21 16:00:00 2019-07-23 04:00:00 13097     
 12 HNL    2018-08-21 16:00:00 2019-07-23 04:00:00 13095     
 13 HTG    2018-08-2

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)